In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install split-folders tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import tensorflow as tf
import numpy as np
import splitfolders
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [4]:
base_dir = '/content/drive/MyDrive/Dataset'

Split Dataset into 3 part: Train, Val, and Test

In [5]:
splitfolders.ratio(
    base_dir,
    output='Capstone',
    ratio=(.7, 0.2,0.1)
)

Copying files: 2375 files [02:40, 14.83 files/s]


In [6]:
train_dir = 'Capstone/train'
val_dir = 'Capstone/val' 
test_dir = 'Capstone/test'

os.listdir(train_dir)

['Healthy', 'Hispa', 'BrownSpot', 'LeafBlast']

In [7]:
class_name = ['Healthy', 'Hispa', 'BrownSpot', 'LeafBlast']

Data Augmentation

In [8]:
train_datagen = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=0.45, 
                    width_shift_range=0.2,
                     height_shift_range=0.2,
                     zoom_range=(0.2), 
                     fill_mode='nearest', 
                    horizontal_flip=True,
                    )

test_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

In [9]:
train_generator = train_datagen.flow_from_directory(
    train_dir, 
    target_size=(150,150), 
    batch_size=32, 
    shuffle=True,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    val_dir,
    target_size=(150,150),
    batch_size=32,
    shuffle=True,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150,150),
    batch_size=32,
    shuffle=True,
    class_mode='categorical'
)

Found 1661 images belonging to 4 classes.
Found 474 images belonging to 4 classes.
Found 240 images belonging to 4 classes.


Modeling

In [10]:
from keras.callbacks import EarlyStopping
monitor_val_acc = EarlyStopping(monitor='val_accuracy', patience=3)

In [11]:
from tensorflow.keras.applications import InceptionV3
tfmodel = InceptionV3(weights='imagenet', 
                                include_top=False, 
                                input_shape=(150,150,3))

87910968/87910968 [==============================] - 4s 0us/step


In [12]:
from keras.models import Sequential
from keras import optimizers, losses, activations, models
from keras.layers import Convolution2D, Dense, Input, Flatten, Dropout, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D

In [23]:
model = Sequential()
model.add(tfmodel)
model.add(GlobalAveragePooling2D())
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.4))      
model.add(Dense(16, activation='relu'))
model.add(Dense(4, activation='softmax'))

model.compile(loss='categorical_crossentropy', 
              optimizer=optimizers.Adam(),
              metrics=['accuracy'])

In [24]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 3, 3, 2048)        21802784  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 flatten_5 (Flatten)         (None, 2048)              0         
                                                                 
 dense_17 (Dense)            (None, 512)               1049088   
                                                                 
 dense_18 (Dense)            (None, 256)               131328    
                                                                 
 dropout_5 (Dropout)         (None, 256)               0         
                                                      

In [25]:
history = model.fit(train_generator,
                    epochs=50,
                    callbacks=[monitor_val_acc],
                    validation_data=validation_generator)

Epoch 1/50
52/52 [==============================] - 731s 14s/step - loss: 1.4311 - accuracy: 0.2836 - val_loss: 9094.7451 - val_accuracy: 0.2194
Epoch 2/50
52/52 [==============================] - 722s 14s/step - loss: 1.3909 - accuracy: 0.2529 - val_loss: 94.8750 - val_accuracy: 0.3059
Epoch 3/50
52/52 [==============================] - 720s 14s/step - loss: 1.3841 - accuracy: 0.2986 - val_loss: 13.3705 - val_accuracy: 0.2932
Epoch 4/50
52/52 [==============================] - 719s 14s/step - loss: 1.3893 - accuracy: 0.2830 - val_loss: 1.9923 - val_accuracy: 0.2932
Epoch 5/50
52/52 [==============================] - 723s 14s/step - loss: 1.3818 - accuracy: 0.2896 - val_loss: 2.7817 - val_accuracy: 0.2932
